## Mount google drive to pull the input data


#About

In this notebook we are using Bayesian Optimization to select the most promising values for the hyper parameters of our [neural network model](https://github.com/samirsebbah/mec-mini-projects/blob/master/LeapSimModeling.ipynb).

Fine-tuning these hyperparameters can significantly impact the network's ability to generalize and perform well on unseen data.

Below are the hyperparameters we considered during the tuning process:
1. Learning Rate
2. Batch Size
3. Number of Epochs
4. Number of hidden layers (2 layers is the best option). Removed
5. Number of neurons per Layer
6. Activation function

From the optimization below the best parameters' values are
Best hyperparameters: [0.0008468986729585612, 80, 50, 369, 2, 'relu']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import libs and setup parameters

In [ ]:
%%capture

import numpy as np
import pandas as pd
import polars as pl

import os
import time

from numpy import random

import matplotlib.pyplot as plt
import seaborn as sns


# configs and params
MY_SEED = 43
TRAINING_SAMPLE_NBR_ROWS =1_000_000
MY_COLORS = ('darkblue','darkgreen','darkred')

pd.set_option('display.max_columns', None) # display all columns


# Set Google Drive as working directory.

# the base Google Drive directory
root_dir = "/content/drive/MyDrive/"
# location for the project files to be saved
project_working_dir = "leap_sim_project"

# input folder of dataset
dataset_input_zip_folder = root_dir + '/Kaggle/leap-atmospheric-physics-ai-climsim.zip'
train_data_file_name = 'train.csv'
test_data_file_name = 'test.csv'
submission_data_file_name = 'sample_submission.csv'

## Helper methods

In [ ]:
from zipfile import ZipFile as zipfile

def create_and_set_working_directory(project_folder):
    # check if your project folder exists. if not, it will be created.
    if os.path.isdir(root_dir + project_folder) == False:
        os.mkdir(root_dir + project_folder)
        print(root_dir + project_folder + ' did not exist but was created.')

    # change the OS to use your project folder as the working directory
    os.chdir(root_dir + project_folder)

create_and_set_working_directory(project_working_dir)

# method to read a dataset from a zipped folder
def read_dataset(dataset_input_zip_folder, train_data_file_name, nbr_rows=1, cols=None):
    with zipfile(dataset_input_zip_folder) as z:
        with z.open(train_data_file_name) as f:
            df = pd.read_csv(f, nrows = nbr_rows, usecols=cols)
    return df

# Composit features
In this section composit features are isolated.


1.   The variables with multiple dimensions are referred to as composit features. Each composit feature is composed of its 60 dimensional features
2.   The scaler variables are grouped together and referred to as scaler feature.

In [ ]:
# sample the data
import re

# parse columns' names of the input/output variables
# input features are categorized as composit and scaler
# composit ones are the with multiple dimensions
# the scaler ones are grouped together

df_submission_columns = read_dataset(dataset_input_zip_folder, submission_data_file_name, 0)

targets = df_submission_columns.columns.to_list()
targets.remove('sample_id')
target_scalers_cols = targets
target_composits_cols = list()

target_composits = "ptend_t,ptend_q0001,ptend_q0002,ptend_q0003,ptend_u,ptend_v".split(",")

all_elts_of_composit_targets = dict()
for pattern in target_composits:
    all_elts_of_composit_targets[pattern] = [x for x in targets if re.match(pattern + "_[0-9]+$",x)]
    target_scalers_cols = [x for x in target_scalers_cols if x not in all_elts_of_composit_targets[pattern]]
    target_composits_cols.extend(all_elts_of_composit_targets[pattern]) # keep a list of all columns

# features
df_train_columns = read_dataset(dataset_input_zip_folder, train_data_file_name, 0)

features = [x for x in df_train_columns.columns.tolist() if x not in ['sample_id']+targets]
feature_scalers_cols = features
feature_composits_cols = list()

feature_composits = "state_t,state_q0001,state_q0002,state_q0003,state_u,state_v,pbuf_ozone,pbuf_CH4,pbuf_N2O".split(",")

all_elts_of_composit_features = dict()
for pattern in feature_composits:
    all_elts_of_composit_features[pattern] = [x for x in features if re.match(pattern + "_[0-9]+$",x)]
    feature_scalers_cols = [x for x in feature_scalers_cols if x not in all_elts_of_composit_features[pattern]]
    feature_composits_cols.extend(all_elts_of_composit_features[pattern])

# Import PCs and target variables
In the EDA, we transformed the inupt data into principal components. Below the PCs are loaded from parquet files

In [ ]:
def load_data (root_dir, project_dir, file_name, feature_name):
    df = pd.read_parquet(root_dir + project_dir + '/' + file_name)
    df.columns = [feature_name + '_' + str(col) for col in df.columns]
    return df


df_train = dict()
for feature_name in feature_composits:
    print("Loading pcs of composit feature ", feature_name)
    df_train[feature_name] = load_data(root_dir, project_working_dir, 'composit_feature_' + feature_name + '_df.parquet', feature_name)


df_train["scaler"] = load_data(root_dir, project_working_dir, 'scaler_features_df.parquet', 'scaler')

df_output = load_data(root_dir, project_working_dir, 'outputs_df.parquet', '')



Loading pcs of composit feature  state_t
Loading pcs of composit feature  state_q0001
Loading pcs of composit feature  state_q0002
Loading pcs of composit feature  state_q0003
Loading pcs of composit feature  state_u
Loading pcs of composit feature  state_v
Loading pcs of composit feature  pbuf_ozone
Loading pcs of composit feature  pbuf_CH4
Loading pcs of composit feature  pbuf_N2O


# Modeling Approach

First define the custom R2 metric to measure the performance of the model

In [ ]:
import tensorflow as tf

# Define custom R-squared metric
class R2Metric(tf.keras.metrics.Metric):
    def __init__(self, name='r_squared', **kwargs):
        super(R2Metric, self).__init__(name=name, **kwargs)
        self.sse = self.add_weight(name='sse', initializer='zeros')
        self.sst = self.add_weight(name='sst', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        # Residual sum of squares (sse)
        sse = tf.reduce_sum(tf.square(y_true - y_pred))

        # Total sum of squares (sst)
        sst = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))

        # Update the states
        self.sse.assign_add(sse)
        self.sst.assign_add(sst)

    def result(self):
        return 1 - (self.sse / (self.sst + tf.keras.backend.epsilon()))  # To prevent division by zero

    def reset_states(self):
        self.sse.assign(0.0)
        self.sst.assign(0.0)



## The Multi-Layer Perceptron (MLP) Model

# Optimization

In [ ]:
!pip install scikit-optimize > /dev/null

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input, Dense, Concatenate, Activation

from tensorflow.keras.optimizers import Adam
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.layers import BatchNormalization



state_t_input = df_train['state_t'].to_numpy()
state_q0001_input = df_train['state_q0001'].to_numpy()
state_q0002_input = df_train['state_q0002'].to_numpy()
state_q0003_input = df_train['state_q0003'].to_numpy()
state_u_input = df_train['state_u'].to_numpy()
state_v_input = df_train['state_v'].to_numpy()
pbuf_ozone_input = df_train['pbuf_ozone'].to_numpy()
pbuf_CH4_input = df_train['pbuf_CH4'].to_numpy()
pbuf_N2O_input = df_train['pbuf_N2O'].to_numpy()
scaler_input = df_train['scaler'].to_numpy()
output_input = df_output.to_numpy()


# Split Data into Training and Validation Sets
state_t_train,state_t_val,\
state_q0001_train,state_q0001_val,\
state_q0002_train,state_q0002_val,\
state_q0003_train,state_q0003_val,\
state_u_train,state_u_val,\
state_v_train,state_v_val,\
pbuf_ozone_train,pbuf_ozone_val,\
pbuf_CH4_train,pbuf_CH4_val,\
pbuf_N2O_train,pbuf_N2O_val,\
scaler_train,scaler_val,output_train,output_val = train_test_split(state_t_input,\
                   state_q0001_input,\
                   state_q0002_input,\
                   state_q0003_input,\
                   state_u_input,\
                   state_v_input,\
                   pbuf_ozone_input,\
                   pbuf_CH4_input,\
                   pbuf_N2O_input,\
                   scaler_input,\
                   output_input,\
                   test_size=0.2, random_state=42)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# define learning rate reduction on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)



In [ ]:

# 9. Train the model with validation data
def fit_model2(model, nbr_epochs=2, batch_size=32):
    history = model.fit([state_t_train,\
                     state_q0001_train,\
                     state_q0002_train,\
                     state_q0003_train,\
                     state_u_train,\
                     state_v_train,\
                     pbuf_ozone_train,\
                     pbuf_CH4_train,\
                     pbuf_N2O_train,\
                     scaler_train], output_train,\
                    validation_data=([state_t_val,\
                     state_q0001_val,\
                     state_q0002_val,\
                     state_q0003_val,\
                     state_u_val,\
                     state_v_val,\
                     pbuf_ozone_val,\
                     pbuf_CH4_val,\
                     pbuf_N2O_val,\
                     scaler_val], output_val),
                    epochs=nbr_epochs, batch_size=batch_size, callbacks=[early_stopping])

    val_loss, val_mse = model.evaluate([state_t_val,\
                     state_q0001_val,\
                     state_q0002_val,\
                     state_q0003_val,\
                     state_u_val,\
                     state_v_val,\
                     pbuf_ozone_val,\
                     pbuf_CH4_val,\
                     pbuf_N2O_val,\
                     scaler_val], output_val)
    return val_loss


# Create a function to define and train a neural network model
def create_and_train_nn(params):

  inputs = dict()
  for feature_name in df_train.keys():
    inputs[feature_name] = Input(shape =(df_train[feature_name].shape[1],), name=feature_name)

  # Define the uniform initializer of the weights
  initializer = RandomUniform(minval=-0.1, maxval=0.1)


  # input layer
  input_layer = Concatenate()([inputs['state_t'],inputs['state_q0001'],inputs['state_q0002'],inputs['state_q0003'],inputs['state_u'],inputs['state_v'],inputs['pbuf_ozone'],inputs['pbuf_CH4'],inputs['pbuf_N2O'],inputs['scaler']])

  # batch normalization layer
  batch_norm_layer = BatchNormalization()(input_layer)

  # 2 hidden layers
  hidden_layer = Dense(units=params[3], activation=params[5], kernel_initializer=initializer)(batch_norm_layer)
  hidden_layer = Dense(units=params[3] + len(feature_scalers_cols), activation=params[5], kernel_initializer=initializer)(hidden_layer)

  # another batch normalization layer
  #batch_norm_layer = tf.keras.layers.BatchNormalization()(hidden_layer)


  # output layer
  output_layer = Dense(units=df_output.shape[1], activation='linear', kernel_initializer=initializer)(hidden_layer)


  # 7. Create the model
  model = Model(inputs=[inputs[feature_name] for feature_name in df_train.keys()], outputs=output_layer)

  # Compile the model with the custom R-squared metric
  # tried adagrad but was not as good as adam
  optimizer = Adam(learning_rate=params[0])
  model.compile(optimizer=optimizer, loss='mse', metrics=[R2Metric()])
  return model



def create_and_train_nn2(params):
  model = create_and_train_nn(params)
  return fit_model2(model, params[1], params[2])

# Define the hyperparameter search space
search_space = [
    #Integer(1, 3, name='num_hidden_layers'),  # Number of hidden layers (1-3)
    Real(1e-4, 1e-2, "log-uniform", name='learning_rate'),  # Learning rate (log scale)
    #Real(0.0, 0.5, name='dropout_rate'),  # Dropout rate (0-50%)
    Integer(50, 80, name='batch_size'),  # Batch size (16-128)
    Integer(30, 50, name='epochs'), # Number of epochs (10-50)
    Integer(df_output.shape[1], (df_output.shape[1]+len(feature_scalers_cols)), name='num_neurons'),  # Number of neurons per layer (32-256)
    Integer(0, 2, name='num_hidden_layers'),  # Number of hidden layers (1-3),
    Categorical(['relu', 'tanh', 'sigmoid'], name='activation')
]

def gpu2():
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print(
        '\n\nThis error most likely means that this notebook is not '
        'configured to use a GPU.  Change this in Notebook Settings via the '
        'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise SystemError('GPU device not found')

  with tf.device('/device:GPU:0'):
    # Perform the optimization using Bayesian Optimization (GP-based optimization)
    result = gp_minimize(
      func=create_and_train_nn2,  # Objective function
      dimensions=search_space,  # Hyperparameter space
      n_calls=20,  # Number of iterations
      random_state=42,  # Random state for reproducibility
      verbose=True
    )

    # Get the best hyperparameters and print them
    best_hyperparameters = result.x
    print("Best hyperparameters:", best_hyperparameters)

#cpu()
gpu2()


Iteration No: 1 started. Evaluating function at random point.
Epoch 1/56
17392/17392 ━━━━━━━━━━━━━━━━━━━━ 65s 3ms/step - loss: 20.9641 - r_squared: 0.9689 - val_loss: 6.1217 - val_r_squared: 0.9910
Epoch 2/56
17392/17392 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 8.8735 - r_squared: 0.9870 - val_loss: 5.9489 - val_r_squared: 0.9913
Epoch 3/56
17392/17392 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 8.0636 - r_squared: 0.9882 - val_loss: 5.7701 - val_r_squared: 0.9915
Epoch 4/56
17392/17392 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 7.6579 - r_squared: 0.9887 - val_loss: 6.1128 - val_r_squared: 0.9910
Epoch 5/56
17392/17392 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 7.4757 - r_squared: 0.9890 - val_loss: 6.4441 - val_r_squared: 0.9906
Epoch 6/56
17392/17392 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 7.3407 - r_squared: 0.9892 - val_loss: 5.9315 - val_r_squared: 0.9913
Epoch 7/56
17392/17392 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 7.1881 - r_squared: 0.9894 - val_loss: 5.4151 - val_r_squared: